## 시계열(Time Series) 데이터
- 시간의 흐름에 따라 변화하는 데이터를 의미함
- 날짜/시간이 인덱스 역할을 하거나 분석의 중심 기준이 되는 데이터
- ex) 주가 데이터, 기온 데이터, 로그 기록, 매출 기록 등

In [5]:
# 샘플 데이터
data = '''
date,temperature
2025-11-15,18
2025-11-16,19
2025-11-17,10
2025-11-18,7
2025-11-19,9
2025-11-20,15
2025-11-21,13
2025-11-22,15
2025-11-23,16
2025-11-24,17
'''

In [8]:
from io import StringIO
import pandas as pd
# StringIO() : 문자열 내용을 파일 객체로 변환
sample_csv = StringIO(data)

# csv 형태의 데이터를 DataFrame 으로 변환
df = pd.read_csv(sample_csv)
df

,date,temperature
0,2025-11-15,18
1,2025-11-16,19
2,2025-11-17,10
3,2025-11-18,7
4,2025-11-19,9
5,2025-11-20,15
6,2025-11-21,13
7,2025-11-22,15
8,2025-11-23,16
9,2025-11-24,17


In [11]:
print('* 변환 전 컬럼 타입 *')
print(df.dtypes)
# * 날짜 컬럼의 데이터 타입을 datetime 타입으로 변환
#   => to_datetime() : 날짜 타입으로 변환

df['date'] = pd.to_datetime(df['date'])
print('* 변환 후 컬럼 타입 *')
print(df.dtypes)

* 변환 전 컬럼 타입 *
date           datetime64[ns]
temperature             int64
dtype: object
* 변환 후 컬럼 타입 *
date           datetime64[ns]
temperature             int64
dtype: object


In [16]:
df

# 날짜 컬럼을 인덱스로 설정 => 시계열 분석 시 기본 사항
#   set_index(컬럼명) : 데이터프레임에서 특정 컬럼을 인덱스로 변경하고자 할때 사용하는 함수
df = df.set_index('date')
df

KeyError: "None of ['date'] are in the columns"

In [17]:
df

,temperature
date,
2025-11-15,18
2025-11-16,19
2025-11-17,10
2025-11-18,7
2025-11-19,9
2025-11-20,15
2025-11-21,13
2025-11-22,15
2025-11-23,16


In [23]:
# * 특정 기간 필터링 ( 2025-11-17 ~ 2025-11-21)
# df[2:7]
# df.loc['2025-11-17' : '2025-11-21']
df['2025-11-17' : '2025-11-21']
# => 인덱스 타입이 DatetimeIndex 인 경우, 행 슬라이싱 가능! loc를 사용하는 것 과 동일
# df.index


,temperature
date,
2025-11-17,10
2025-11-18,7
2025-11-19,9
2025-11-20,15
2025-11-21,13


In [26]:
# resample(rule) : DatetimeIndex 기반의 시계열 데이터를 특정 시간 단위로 새롭게 그룹화 하는 기능
# - rule : 리샘플링 단위 지정
#       'Y' : 연말, 'YS' : 연초
#       'M' : 월말, 'MS' : 월초
#       'W' : 주말 기준 주 단위
#       'D' : 일별, 'H' : 시간, 'min'/ 'T' : 분, 'S' : 초
#       'Q' : 분기 말, 'QS' : 분기 시작일

# 샘플링 유형
#   - 다운 샘플링 : 작은 주기 -> 큰 주기로 변경 (일 -> 월)
#   - 업 샘플링   : 큰 주기 -> 작은 주기로 변경 (월 -> 일)

# * 월별 평균 리샘플링 (resample)
monthly_mean = df.resample('ME').mean()   # 'M' == 'ME' ('M' 곧 사라질듯)
monthly_mean

,temperature
date,
2025-11-30,13.9


In [ ]:
# rolling(window) : 슬라이딩 윈도우 방식으로 일정 구간을 이동시키며 계산하는 함수
#   - 슬라이딩 윈도우 : 고정된 크기의 구간(window)을 한 칸씩 이동시키며 반복계산하는 기법
#   - 이동 평균 계산 시 mean()와 함께 사용
#   - 그 외, 최근 3일 최대 값, 최근 7일 합계 등 특정 구간을 기준으로 통계값을 계산

# * 3일 이동 평균
df['temp_mean_3'] = df['temperature'].rolling(window=3).mean()
df
# => 3개의 데이터가 쌓여야 계산되므로, 15일, 16일 데이터는 NaN으로 표시됨.

# --> 결측치 없이 계산하고자 할 경우 min_periods 옵션 사용
df['temp_mean_3'] = df['temperature'].rolling(window=3, min_periods=1).mean()
df
# - min_periods : 계산을 수행하기 위해 필요한 최소 데이터 개수

,temperature,temp_mean_3
date,,
2025-11-15,18,18.000000
2025-11-16,19,18.500000
2025-11-17,10,15.666667
2025-11-18,7,12.000000
2025-11-19,9,8.666667
2025-11-20,15,10.333333
2025-11-21,13,12.333333
2025-11-22,15,14.333333
2025-11-23,16,14.666667
